### References

* Chatbot with TF: https://chatbotsmagazine.com/contextual-chat-bots-with-tensorflow-4391749d0077
* Good reference list for reading: http://www.wildml.com/2016/04/deep-learning-for-chatbots-part-1-introduction/
* **Chameleon paper:**http://www.cs.cornell.edu/~cristian/papers/chameleons.pdf
* **Pandas pipelines:**https://medium.com/bigdatarepublic/integrating-pandas-and-scikit-learn-with-pipelines-f70eb6183696


### Options:
* Use categories from semtype tagging to identify

## Initialize

In [8]:
# Utility
import sys,os
import time

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500) # more columns displayed at once
pd.options.display.max_colwidth = 200 # more of the text displayed at once

# Custom
from processing import tag_utterances
from processing import load_sem_types

## NLP
import spacy

In [1]:
# UMLS
# Set absolute path the QuickUMLS Server
abs_path_umls = '/Users/austinpowell/Google_Drive/kp_datascience/doctor_notes/ontology/UMLS'
abs_path_data_umls = '/Users/austinpowell/Google_Drive/kp_datascience/doctor_notes/ontology/UMLS/QuickUMLS_db'
sys.path.append(abs_path_umls+'/QuickUMLS')
from quickumls import QuickUMLS
tagger = QuickUMLS(abs_path_data_umls)

In [31]:
df.columns.tolist()

['body',
 'score_hidden',
 'archived',
 'name',
 'author',
 'author_flair_text',
 'downs',
 'created_utc',
 'subreddit_id',
 'link_id',
 'parent_id',
 'score',
 'retrieved_on',
 'controversiality',
 'gilded',
 'id',
 'subreddit',
 'ups',
 'distinguished',
 'author_flair_css_class',
 'removal_reason',
 'is_clinician',
 'tokenized_sents']

In [2]:
path_to_data = '../data/reddit_comments_askDocs_2014_to_2018_03.gz'
df = pd.read_csv(path_to_data,dtype={'body':str,'score_hidden':float})
print('Shape',df.shape)
df.head(2)

Shape (557648, 21)


/Users/austinpowell/Google_Drive/kp_datascience/virtual_envs/nlp-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
0,"for a manlet such as yourself I'd recommend at least 70 oz of water daily and at least 7 hours of sleep. Cut down on red meat, smoking, and sodium intake and check back in a few days.",NaN,NaN,NaN,-Ai,This user has not yet been verified.,NaN,1513411674,t5_2xtuc,t3_7k5x2h,t3_7k5x2h,0,1.514772e+09,0,0,drbt2db,AskDocs,NaN,NaN,default,NaN
1,Thank you very much for answering!,NaN,NaN,NaN,-SY,This user has not yet been verified.,NaN,1445798103,t5_2xtuc,t3_3q697b,t1_cwcf958,2,1.447190e+09,0,0,cwcfjpr,AskDocs,2.0,NaN,default,NaN


(1136, 22)

In [69]:
print(len(df['link_id'].unique())/df.shape[0])
print(len(df['id'].unique())/df.shape[0])
print(len(df['parent_id'].unique())/df.shape[0])


0.2661284537916392
0.9484190743981866
0.7033110492640519


## Data Dictionary:
Original post can be seen with the link_id
* **link_id** Link to the page where the original thread was started
    * **id** A posts id. The only reason this may not be unique
    * **parent_id** These ids are pointing back towards the original post and represent a new comment that is replying directly towards the original posting. Can be considered a general comment. I.e. All posts with the same parent_id (following the "_") as the link_id are. E.g. If the link_id is "t3_827pgt", all parent_id's with "827pgt" are pointing towards that original post.

* **subreddit_id** and **subreddit** Irrelevant


### Documentation:
* Figuring out how to organize a reply and response is tricky:
    * On one hand, the people asking the questions should be the non-clinician folks and the clinicians answering the questions. But it is a community forum so people can chime in with ''answers'' whenever. Ansd they might be correct.
     *  Also, it is likely that clinicians will ask follow-up questions if it is more debate than answer so perhaps drilling down into a solution
     * This latest point gets at how to utilize the id's and parent_ids

In [71]:
print('Number of rows:',df.shape[0])
print('Number of unique posters:',len(df.author.unique()))
print()
print('Number of link_ids:',len(df.link_id.unique()))
print('Number of parent_ids:',len(df.parent_id.unique()))
print('Number of ids:',len(df.id.unique()))

Number of rows: 556824
Number of unique posters: 77146

Number of link_ids: 148328
Number of parent_ids: 391731
Number of ids: 528095


In [98]:
pdf.loc[df.parent_id.str.contains('827pgt',na=False)]rint(df.loc[df.link_id.str.contains('t3_827pgt',na=False)].shape)
print(df.loc[df.link_id.str.contains('827pgt',na=False)].shape)

print(.shape)

(44, 22)
(44, 22)
(11, 22)


In [74]:
print(df.loc[df.link_id.str.contains('827pgt',na=False)].shape)
print(len(df.loc[df.link_id.str.contains('827pgt',na=False)]['id'].unique()))
print(df.loc[df.parent_id.str.contains('827pgt',na=False)].shape)
print(len(df.loc[df.parent_id.str.contains('827pgt',na=False)]['id'].unique()))

(44, 22)
44
(11, 22)
11


In [107]:
df.loc[df.parent_id.str.contains('827pgt',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
47080,"I'm not a doctor, but do you guys think this is Ankylosing Spondylitis?",NaN,NaN,NaN,Yossi25,This user has not yet been verified.,NaN,1522308033,t5_2xtuc,t3_827pgt,t3_827pgt,1,1.525707e+09,0,0,dwgoznj,AskDocs,NaN,NaN,default,NaN,827pgt
59878,"Degenerative issues run in my family. From a young age, I was showing signs of DDD and it has progressed as the years go on. I wouldn't call my case severe, but enough to where I had spent a solid...",NaN,NaN,NaN,vanteal,This user has not yet been verified.,NaN,1520299064,t5_2xtuc,t3_827pgt,t3_827pgt,5,1.524857e+09,0,0,dv8rpap,AskDocs,NaN,NaN,default,NaN,827pgt
154535,[deleted],NaN,NaN,NaN,[deleted],NaN,NaN,1520316050,t5_2xtuc,t3_827pgt,t3_827pgt,1,1.524865e+09,0,0,dv95z1b,AskDocs,NaN,NaN,NaN,NaN,827pgt
189543,Im so sorry for your loss. I’m not a doctor so please don’t take this as fact as it’s based on my personal opinion and experience. My last MRI was pretty similar and I’ve been in pain 24 hours a d...,NaN,NaN,NaN,twinkie45,This user has not yet been verified.,NaN,1520287643,t5_2xtuc,t3_827pgt,t3_827pgt,6,1.524851e+09,0,0,dv8go5a,AskDocs,NaN,NaN,default,NaN,827pgt
195752,"Sorry for your loss. It is never easy, she had a reason.\nStill leaves a lot of pain behind.\n\nI have degeneration in my neck but it is nothing like what your mother may have had. As I recall the...",NaN,NaN,NaN,DennyBenny,This user has not yet been verified.,NaN,1520297740,t5_2xtuc,t3_827pgt,t3_827pgt,2,1.524856e+09,0,0,dv8qg82,AskDocs,NaN,NaN,default,NaN,827pgt
253902,"It's not remotely the same but my father was about the same age as your mother, and in the end he chose not to go back to the hospital to treat his COPD, choosing to die at the seniors centre he w...",NaN,NaN,NaN,hobbitlover,NaN,NaN,1520298169,t5_2xtuc,t3_827pgt,t3_827pgt,6,1.524856e+09,0,0,dv8que6,AskDocs,NaN,NaN,NaN,NaN,827pgt
254645,She had degenerative disc and thinning disc from poor posture and old age. See pic: https://www-epainassist-com.cdn.ampproject.org/ii/w1000/s/www.epainassist.com/images/can-you-get-degenerative-di...,NaN,NaN,NaN,jbjbjb55555,NaN,NaN,1520310309,t5_2xtuc,t3_827pgt,t3_827pgt,-2,1.524862e+09,1,0,dv91xtu,AskDocs,NaN,NaN,NaN,NaN,827pgt
348588,"Her fears of further degeneration and living in pain, would be My guess. Im sorry for your loss.",NaN,NaN,NaN,MDFrankensteen,This user has not yet been verified.,NaN,1520274122,t5_2xtuc,t3_827pgt,t3_827pgt,45,1.524842e+09,0,0,dv81ye0,AskDocs,NaN,NaN,default,NaN,827pgt
390103,that looks alot like my MRI. I also have ddd. It's pretty painful and at times I have thought I didn't want to continue all the way out to the end of the disease because my pain is usually at leas...,NaN,NaN,NaN,molotavcocktail,This user has not yet been verified.,NaN,1520279487,t5_2xtuc,t3_827pgt,t3_827pgt,18,1.524845e+09,0,0,dv87o36,AskDocs,NaN,NaN,default,NaN,827pgt
405258,So sorry for your loss.\n\nRoughly read through the report and from what I'm able to interpret it seems like your Mother is suffering from severe degeneration of the spine which resulted in multip...,NaN,NaN,NaN,Sgmedicalstudent,NaN,NaN,1520274423,t5_2xtuc,t3_827pgt,t3_827pgt,81,1.524842e+09,0,0,dv829t2,AskDocs,NaN,NaN,NaN,NaN,827pgt


In [110]:
df.loc[df.parent_id.str.contains('dv8rpap',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
223990,"Wow, thank you for your story. I do not believe my mother was a coward in the slightest. She was a very strong woman and she did what she had to do. I’m not struggling against the grief, I’m float...",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520302687,t5_2xtuc,t3_827pgt,t1_dv8rpap,5,1.524859e+09,0,0,dv8v4ny,AskDocs,NaN,NaN,NaN,NaN,dv8rpap


In [82]:
unique_ids = df.id.unique()
df.loc[df['id'].isin(unique_ids)].head()

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,is_clinician
0,"for a manlet such as yourself I'd recommend at least 70 oz of water daily and at least 7 hours of sleep. Cut down on red meat, smoking, and sodium intake and check back in a few days.",NaN,NaN,NaN,-Ai,This user has not yet been verified.,NaN,1513411674,t5_2xtuc,t3_7k5x2h,t3_7k5x2h,0,1.514772e+09,0,0,drbt2db,AskDocs,NaN,NaN,default,NaN,0
1,Thank you very much for answering!,NaN,NaN,NaN,-SY,This user has not yet been verified.,NaN,1445798103,t5_2xtuc,t3_3q697b,t1_cwcf958,2,1.447190e+09,0,0,cwcfjpr,AskDocs,2.0,NaN,default,NaN,0
2,Never been tested for that. I was hoping the full blood test would reveal a lot of things. I am gonna find a good doctor and I will ask about it. And yeah other than the myocarditis. My heart ...,NaN,NaN,NaN,-o2,This user has not yet been verified.,NaN,1461952470,t5_2xtuc,t3_4gz1fi,t1_d2mbnhi,1,1.463777e+09,0,0,d2mce34,AskDocs,1.0,NaN,default,NaN,0
3,"She said her constant abdominal pain is a 6, the pain during sex was a 9 (we had to stop immediately). No fever.",NaN,NaN,NaN,05P,This user has not yet been verified.,NaN,1504214332,t5_2xtuc,t3_6x9jk0,t1_dme9bp9,1,1.504553e+09,0,0,dme9lzr,AskDocs,NaN,NaN,default,NaN,0
4,"She's had it for 8 months, we've never had any issues with it before.",NaN,NaN,NaN,05P,This user has not yet been verified.,NaN,1504217835,t5_2xtuc,t3_6x9jk0,t1_dmebxgt,1,1.504554e+09,0,0,dmecohs,AskDocs,NaN,NaN,default,NaN,0


In [111]:
df.loc[df.parent_id.str.contains('dv8v4ny',na=False)]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
226939,"I can’t help with your particular question, but I just wanted to say that you seem like a very compassionate, kind, and strong person, and I really wish you the best. I’m so sorry for your loss.",NaN,NaN,NaN,pegmatitic,This user has not yet been verified.,NaN,1520306013,t5_2xtuc,t3_827pgt,t1_dv8v4ny,5,1.524860e+09,0,0,dv8y923,AskDocs,NaN,NaN,default,NaN,dv8v4ny


In [90]:
df.loc[df.parent_id.str.contains('t1_dv81ye0',na=False)].loc[df['author']=='kiwikellie']

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,pd
223992,Thank you very much. I'm not wondering why she did what she did. I absolutely understand why. I'm just looking for answers on what was wrong with her spine.,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274235,t5_2xtuc,t3_827pgt,t1_dv81ye0,9,1.524842e+09,0,0,dv822m9,AskDocs,NaN,NaN,NaN,NaN,dv81ye0


In [3]:
df.loc[df.link_id.str.contains('t3_827pgt',na=False)].iloc[:10]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
9,Laying here due to severe Sciatica and Meralgia paresthetica. I seriously sympathize and empathize your pain. Bless you.,NaN,NaN,NaN,0MY,This user has not yet been verified.,NaN,1520284462,t5_2xtuc,t3_827pgt,t1_dv87o36,8,1.524848e+09,0,0,dv8d5po,AskDocs,NaN,NaN,default,NaN
7937,"Which isnt a disease, its a misnomer. We all have DDD but some worse than others.",NaN,NaN,NaN,Jae_t,This user has not yet been verified.,NaN,1520286469,t5_2xtuc,t3_827pgt,t1_dv82n3m,-5,1.524850e+09,1,0,dv8fd0y,AskDocs,NaN,NaN,default,NaN
42816,"Yes, unfortunately. It seems she had [myelomalacia](https://en.wikipedia.org/wiki/Myelomalacia) at the level around the third cervical vertebra (C3), which would eventually lead to paralysis of ev...",NaN,NaN,NaN,Phhhhuh,This user has not yet been verified.,NaN,1520357456,t5_2xtuc,t3_827pgt,t1_dv82gtb,3,1.524880e+09,0,0,dv9x79p,AskDocs,NaN,NaN,default,NaN
47080,"I'm not a doctor, but do you guys think this is Ankylosing Spondylitis?",NaN,NaN,NaN,Yossi25,This user has not yet been verified.,NaN,1522308033,t5_2xtuc,t3_827pgt,t3_827pgt,1,1.525707e+09,0,0,dwgoznj,AskDocs,NaN,NaN,default,NaN
59878,"Degenerative issues run in my family. From a young age, I was showing signs of DDD and it has progressed as the years go on. I wouldn't call my case severe, but enough to where I had spent a solid...",NaN,NaN,NaN,vanteal,This user has not yet been verified.,NaN,1520299064,t5_2xtuc,t3_827pgt,t3_827pgt,5,1.524857e+09,0,0,dv8rpap,AskDocs,NaN,NaN,default,NaN
78045,"If left untreated, yes.",NaN,NaN,NaN,Pinky135,"B.S., Medical Lab Sciences",NaN,1520275503,t5_2xtuc,t3_827pgt,t1_dv82gtb,36,1.524843e+09,0,0,dv83f4u,AskDocs,NaN,NaN,verified-mid,NaN
78071,"I can't answer that. It depends on a lot of factors and only the treating doctor can give a treatment plan, if one is possible.",NaN,NaN,NaN,Pinky135,"B.S., Medical Lab Sciences",NaN,1520276404,t5_2xtuc,t3_827pgt,t1_dv840aq,4,1.524843e+09,0,0,dv84dtd,AskDocs,NaN,NaN,verified-mid,NaN
114342,You are going to be an outstanding doctor.,NaN,NaN,NaN,youngmrs,This user has not yet been verified.,NaN,1520318439,t5_2xtuc,t3_827pgt,t1_dv94mjv,11,1.524865e+09,0,0,dv97e6r,AskDocs,NaN,NaN,default,NaN
121124,"Reading through and I’m not a specialist in this field but patients I’ve had with similar impressions:\n\nNo matter what could’ve been done here she would be living with some degree of pain, which...",NaN,NaN,NaN,HokayeZeZ,This user has not yet been verified.,NaN,1520285562,t5_2xtuc,t3_827pgt,t1_dv829t2,18,1.524849e+09,0,0,dv8ed1z,AskDocs,NaN,NaN,default,NaN
132308,"She would almost certainly have been a decent surgical candidate based on the information you have provided. I’m not a spine surgeon, but as an anesthesiologist I take care of people with similar ...",NaN,NaN,NaN,[deleted],NaN,NaN,1520286494,t5_2xtuc,t3_827pgt,t1_dv840aq,29,1.524850e+09,0,0,dv8fe0l,AskDocs,NaN,NaN,NaN,NaN


In [4]:
df.loc[df['author']=='kiwikellie'].iloc[:10]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
223985,Thank you. I believe she was told she would eventually become completely paralyzed. Do you believe that is something this condition could eventually progress to?,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274604,t5_2xtuc,t3_827pgt,t1_dv829t2,35,1.524842e+09,0,0,dv82gtb,AskDocs,NaN,NaN,NaN,NaN
223986,Thank you. It definitely helps to learn as much about her condition as I can.,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520278118,t5_2xtuc,t3_827pgt,t1_dv8643f,14,1.524844e+09,0,0,dv867dt,AskDocs,NaN,NaN,NaN,NaN
223987,"Thank you so much. We all do the best we can with what was handed to us. All the best for you, as well.",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520307281,t5_2xtuc,t3_827pgt,t1_dv8y923,3,1.524861e+09,0,0,dv8zdsb,AskDocs,NaN,NaN,NaN,NaN
223988,Can you please elaborate?,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520288550,t5_2xtuc,t3_827pgt,t1_dv8fd0y,3,1.524851e+09,0,0,dv8hmg0,AskDocs,NaN,NaN,NaN,NaN
223989,I think she was already experiencing such pain and stiffness that the thought that it would get worse and worse was too overwhelming for her to bear. I think she made the best choice for herself.\...,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520288433,t5_2xtuc,t3_827pgt,t1_dv8ed1z,8,1.524851e+09,0,0,dv8hi3n,AskDocs,NaN,NaN,NaN,NaN
223990,"Wow, thank you for your story. I do not believe my mother was a coward in the slightest. She was a very strong woman and she did what she had to do. I’m not struggling against the grief, I’m float...",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520302687,t5_2xtuc,t3_827pgt,t1_dv8rpap,5,1.524859e+09,0,0,dv8v4ny,AskDocs,NaN,NaN,NaN,NaN
223991,"I’m glad to hear that she might have had another option, but also sad that I couldn’t convince her to talk to another doctor. She was so sure that there was nothing that could be done for her.\n\n...",NaN,NaN,NaN,kiwikellie,NaN,NaN,1520287993,t5_2xtuc,t3_827pgt,t1_dv8fe0l,12,1.524851e+09,0,0,dv8h1e9,AskDocs,NaN,NaN,NaN,NaN
223992,Thank you very much. I'm not wondering why she did what she did. I absolutely understand why. I'm just looking for answers on what was wrong with her spine.,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520274235,t5_2xtuc,t3_827pgt,t1_dv81ye0,9,1.524842e+09,0,0,dv822m9,AskDocs,NaN,NaN,NaN,NaN
223993,Thank you for a more detailed explanation.\n\nI’m not upset at my mother for making the choice she did and I don’t accuse her of making a bad choice. She cared for my father through his cancer all...,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520281114,t5_2xtuc,t3_827pgt,t1_dv87o36,24,1.524846e+09,0,0,dv89g6u,AskDocs,NaN,NaN,NaN,NaN
223994,Thank you for such an in-depth explanation. I really appreciate you taking the time to write this for me.\n\nI lost my father when I was 9 and now my mother at 29. I grew up understanding that my ...,NaN,NaN,NaN,kiwikellie,NaN,NaN,1520313546,t5_2xtuc,t3_827pgt,t1_dv930hi,13,1.524864e+09,0,0,dv94al4,AskDocs,NaN,NaN,NaN,NaN


In [6]:
df.loc[df.pd.str.contains('827pgt',na=False)]

AttributeError: 'DataFrame' object has no attribute 'pd'

## Medical Entity Extraction with UMLS

In [3]:
# Load semantic types
sem_type_dict = load_sem_types('../data/SemGroups_2013.txt')

In [4]:
t = tagger.nlp(df['body'].iloc[4])
t.text

"She's had it for 8 months, we've never had any issues with it before."

In [7]:
t = tagger.nlp(t.text)

s = t
matches= tagger.match(s, best_match=True, ignore_syntax=False)
for match in matches:
    dir(match)
    print(match,'\n')
for m in match:
    print(m)

[{'start': 19, 'end': 25, 'ngram': 'months', 'term': 'month', 'cui': 'C1561542', 'similarity': 0.75, 'semtypes': {'T170'}, 'preferred': 1}] 

[{'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C0033213', 'similarity': 0.75, 'semtypes': {'T033'}, 'preferred': 1}, {'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C1706387', 'similarity': 0.75, 'semtypes': {'T170'}, 'preferred': 1}] 

{'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C0033213', 'similarity': 0.75, 'semtypes': {'T033'}, 'preferred': 1}
{'start': 47, 'end': 53, 'ngram': 'issues', 'term': 'issue', 'cui': 'C1706387', 'similarity': 0.75, 'semtypes': {'T170'}, 'preferred': 1}


In [21]:
sd = load_sem_types('../data/SemGroups_2013.txt')


AttributeError: 'set' object has no attribute 'text'

In [11]:
import importlib
from processing import tag_utterances
from processing import load_sem_types

tag_utterances(1, t.text, tagger)

{'T170'}


[[1, 19, 25, 'month', 'C1561542', 0.75, {'Intellectual Product'}]]

In [8]:
df.shape

(557648, 21)

In [9]:
%%time
print('Iterating over every document')
#Iterate over every document and extract the concepts
i=-1        
result = []
for idx,doc in  enumerate(df['body']):

    if idx % 10000 == 0:
        print("Documents processed: {}".format(idx))
    try:
        i+=1
        annotations = tag_utterances(i,doc,tagger)
        result.extend(annotations)
    except Exception as e:
        print(e)
        
df_matches = pd.DataFrame(data=result, columns =['document','start','end','term','cui','similarity','semtypes'])
df_matches.sort_values(by=['document','start'],inplace=True)

Iterating over every document
Documents processed: 0
Documents processed: 10000
Documents processed: 20000
Documents processed: 30000
Documents processed: 40000
Documents processed: 50000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 60000
Documents processed: 70000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 80000
Documents processed: 90000
Documents processed: 100000
Documents processed: 110000
Documents processed: 120000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 130000
Documents processed: 140000
Documents processed: 150000
Documents processed: 160000
Documents processed: 170000
Documents processed: 180000
Documents processed: 190000
Argument 'string' has incorrect type (expected str, got float)
Documents processed: 200000
Argument 'string' has incorrect type (expected str, got float)
Argument 'string' has incorrect type (expected str, got float)
Documents processe

KeyboardInterrupt: 

In [6]:
df.head()

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,...,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason
0,for a manlet such as yourself I'd recommend at...,NaN,NaN,NaN,-Ai,This user has not yet been verified.,NaN,1513411674,t5_2xtuc,t3_7k5x2h,...,0,1.514772e+09,0,0,drbt2db,AskDocs,NaN,NaN,default,NaN
1,Thank you very much for answering!,NaN,NaN,NaN,-SY,This user has not yet been verified.,NaN,1445798103,t5_2xtuc,t3_3q697b,...,2,1.447190e+09,0,0,cwcfjpr,AskDocs,2.0,NaN,default,NaN
2,Never been tested for that. I was hoping the ...,NaN,NaN,NaN,-o2,This user has not yet been verified.,NaN,1461952470,t5_2xtuc,t3_4gz1fi,...,1,1.463777e+09,0,0,d2mce34,AskDocs,1.0,NaN,default,NaN
3,"She said her constant abdominal pain is a 6, t...",NaN,NaN,NaN,05P,This user has not yet been verified.,NaN,1504214332,t5_2xtuc,t3_6x9jk0,...,1,1.504553e+09,0,0,dme9lzr,AskDocs,NaN,NaN,default,NaN
4,"She's had it for 8 months, we've never had any...",NaN,NaN,NaN,05P,This user has not yet been verified.,NaN,1504217835,t5_2xtuc,t3_6x9jk0,...,1,1.504554e+09,0,0,dmecohs,AskDocs,NaN,NaN,default,NaN
